In [85]:
import os
if "/content" in os.getcwd():
  !rm -rf WHere_codes/
  !git clone https://github.com/keenranger/WHere_codes
  os.chdir("/content/WHere_codes")
  !pip install -r requirements.txt

from PDR.modules.DataLoader import *
from PDR.modules.Walker import *
from PDR.modules.PlotlyPlotter import *


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.graph_objects as go

In [86]:
sensor_df = pd.read_csv("data/200713_mag01.csv", names=['experiment', 'time', 'accx', 'accy', 'accz', 'gyrox', 'gyroy',
                                                               'gyroz', 'magx', 'magy', 'magz', 'rot0', 'rot1', 'rot2', 'rot3', 'game0', 'game1', 'game2', 'game3', 'light', "x_uncalib", "y_uncalib","z_uncalib","x_bias","y_bias","z_bias"])
mag_df = sensor_df[['time','magx', 'magy', 'magz',"x_uncalib", "y_uncalib","z_uncalib","x_bias","y_bias","z_bias"]]
mag_df.head()
plot_magnet(mag_df, "before calibration").show()

In [87]:
bias_df = mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']].quantile([0.01, 0.99]).sum()/2
bias_df

x_uncalib    -54.569400
y_uncalib     -8.909400
z_uncalib   -469.192781
dtype: float64

In [88]:
mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']] = mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']][['x_uncalib', 'y_uncalib', 'z_uncalib']] - bias_df
mag_df
plot_magnet(mag_df, "after bias calibration").show()


In [89]:
scale_df = mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']].quantile([0.01, 0.99]).diff()/2
max_num = scale_df.max(axis = 1).max()
scale_df = scale_df/max_num
scale_df.max()


x_uncalib    1.000000
y_uncalib    0.898138
z_uncalib    0.942245
dtype: float64

In [90]:
mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']] = mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']] / scale_df.max()
plot_magnet(mag_df, "after scale calibration")

In [91]:
sensor_df = pd.read_csv("data/200713_mag02.csv", names=['experiment', 'time', 'accx', 'accy', 'accz', 'gyrox', 'gyroy',
                                                               'gyroz', 'magx', 'magy', 'magz', 'rot0', 'rot1', 'rot2', 'rot3', 'game0', 'game1', 'game2', 'game3', 'light', "x_uncalib", "y_uncalib","z_uncalib","x_bias","y_bias","z_bias"])
mag_df = sensor_df[['time','magx', 'magy', 'magz',"x_uncalib", "y_uncalib","z_uncalib","x_bias","y_bias","z_bias"]]
mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']] = mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']][['x_uncalib', 'y_uncalib', 'z_uncalib']] - bias_df
mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']] = mag_df[['x_uncalib', 'y_uncalib', 'z_uncalib']] / scale_df.max()
plot_magnet(mag_df, "after calibration").show()


In [92]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mag_df['time'], 
    y=mag_df['x_uncalib'],
    name = 'x'))
fig.add_trace(go.Scatter(
    x=mag_df['time'], 
    y=mag_df['y_uncalib'],
    name = 'y'))
fig.add_trace(go.Scatter(
    x=mag_df['time'],
    y=mag_df['z_uncalib'],
    name='z'))

In [94]:
magnet_heading = np.arctan2(mag_df['y_uncalib'], mag_df['x_uncalib'])*180/np.pi
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mag_df['time'], 
    y=magnet_heading,
    name = 'heading'))